# Blocking with Word2Vec

In [1]:
import os
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

C:\Users\Riccardo De Cesaris\AppData\Local\Temp\ipykernel_3980\3609187278.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Caricamento dello Schema Mediato da File

In [2]:
schema_mediato = pd.read_csv("./../../datasets/MediatedSchemaSemicolon.csv",sep=";")
print("Dimensioni dello Schema Mediato:", schema_mediato.shape)

Dimensioni dello Schema Mediato: (81706, 6)


In [3]:
records = []
for i in range(0, len(schema_mediato)):
    tokens = []
    vals = schema_mediato.iloc[i].tolist()
    values = list()
    for v in vals:
        if v is not np.nan:
            v = v.lower()
        values.append(v)
        
    if values[0] is not np.nan:
        tokens.extend(values[0].split())
    
    if values[1] is not np.nan:
        elements = values[1].split(',')
        tokens.extend([elem.strip() for elem in elements])
    
    if values[2] is not np.nan:
        tokens.append(values[2])
    
    if values[3] is not np.nan:
        tokens.append(values[3])
    
    if values[4] is not np.nan:
        category = values[4]
        category = category.replace(" &", ",")
        category = category.replace(" /", ",")
        elements = category.split(',')
        tokens.extend([elem.strip() for elem in elements])
    
    if values[5] is not np.nan:
        year_string = values[5]
        if (len(year_string) == 4):
            tokens.append(year_string)
        elif (len(year_string) == 6):
            tokens.append(year_string[:4])
        elif (';' in year_string):
            parts = year_string.split(';')
            first_p = parts[0].split()
            tokens.append(first_p[len(first_p)-1])
    
    records.append(tokens)

In [4]:
import random
final_records = list()
for r in records:
    final_r = r*3
    random.shuffle(final_r)
    final_records.append(final_r)

Osservazione: ogni Record è rappresentato con una Lista di Stringhe, dove le 'stringhe' sono i campi che lo caratterizzano.

## Inizializzazione del Modello "Word2Vec"

In [5]:
num_threads = os.cpu_count()
print("Numero di thread disponibili:", num_threads)

Numero di thread disponibili: 16


In [6]:
model = Word2Vec(sentences=final_records, vector_size=512, window=11, min_count=1, workers=num_threads)

## Calcolo dei Vettori per i Record
Abbiamo settato la dimensione dello Spazio Latente a 512 e per ogni vettore dobbiamo, quindi, ottenere un vettore di 512 valori.

Per un Record il vettore viene ottenuto sommando tra loro "element-wise" i vettori ottenuti dai campi ed infine normalizzando il vettore risultante sul valore 512, pari alla dimensione dello Spazio Latente.

In [7]:
def calculate_vector(record):
    # Inizializza un vettore vuoto
    record_vector = np.zeros(model.vector_size)
    # Somma i vettori di embedding di ciascun token nel record
    for token in record:
        record_vector += model.wv[token]
    #Normalizza il vettore risultante
    record_vector /= 512
    return record_vector

In [8]:
records_w2v = []
for i in range(0, len(records)):
    records_w2v.append(calculate_vector(records[i]))

In [9]:
df = pd.DataFrame(records_w2v)
print("Dimensioni del DataFrame di Vettori:", df.shape)

Dimensioni del DataFrame di Vettori: (81706, 512)


## K-Means sui Vettori per costruire i Blocchi

In [10]:
kmeans = KMeans(n_clusters=143, init='k-means++', random_state=0, tol=0.000001).fit(df)

In [11]:
schema_mediato['block'] = kmeans.labels_

In [12]:
schema_mediato.to_csv("./../../datasets/MediatedSchemaConBlocks_W2V.csv", index=False)